## Functions for automated scripting of our EZMate liquid dispenser

In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline
from scipy import io
import pandas as pd
import numpy as np
import sys 
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import glob
from sklearn import metrics
import seaborn as sns
import getpass
import os
import matplotlib
import re       # regular expressions
from itertools import chain


In [2]:
# 1. select where you are running script:
# needs to be updated using switch?
mac_flag = 0   # 0 for PC users, 1 for MAC users (Tomer)
linux_flag = 1
home_flag = 0
user_name = getpass.getuser() 

# 2. select project name
project_name = 'COVID-19'

# set default values for label sizes for figures:
sns.set_context("paper", rc={'font.size':16, 'axes.titlesize':18, 'axes.labelsize':16, 'xtick.labelsize': 16, 
                                 'ytick.labelsize': 16 })   

In [3]:
if mac_flag:
    MAIN_PATH = '/Users/'
    ROOT_PATH = os.path.join(MAIN_PATH, user_name, 'Dropbox', 'HertzLab')
    sys.path.append('/Users/thertz/Dropbox/HertzLab/Code/Python/Utils/')
elif linux_flag:
    MAIN_PATH = '/home/'
    ROOT_PATH = os.path.join(MAIN_PATH, user_name, 'Dropbox', 'HertzLab')
elif home_flag:
    sys.path.append("C:\\Users\\udish\\Python\\")
    sys.path.append("C:\\Users\\udish\\Desktop\\Ayelet_DB\\Dropbox\\Utils\\")
    ROOT_PATH = os.path.join('C:\\Users\\udish\\Desktop\\Ayelet_DB\\Dropbox')
else:
    ROOT_PATH = os.path.join('//132.72.92.166', 'HertzLab', 'HertzLab') 
    sys.path.append("\\\\132.72.92.166\\HertzLab\\HertzLab\\Code\\Python\\")
    sys.path.append("\\\\132.72.92.166\\HertzLab\\HertzLab\\Code\\Python\\Utils\\")
    # e.g. C:\Users\friedmal\Dropbox\HertzLab\ArrayData\Influenza\obesity\

if home_flag:
    SAVE_PATH = os.path.join('C:\\Users\\udish\\Desktop\\Ayelet_DB\\Dropbox\\', project_name, isotype_name[0] + '_peptide_19')
    FIG_PATH = os.path.join(ROOT_PATH, project_name, 'PaperFigs', '/')
    #SAVE_PATH = os.path.join('C:\\Users\\udish\\', project_name, isotype_name[0] + '_peptide_19')
else:
    SAVE_PATH = os.path.join(ROOT_PATH, 'Code', 'Python', project_name,)     

FIG_PATH = os.path.join(ROOT_PATH, project_name, 'Figs/')
DOC_PATH  = os.path.join(ROOT_PATH, project_name, 'docs')

### load lab specific pacakges (can only be done after paths are set)

In [4]:
import amplotlib as amp
import ezmate_utils as ezutils

In [5]:
# test MD command:
md_cmd = ezutils.generate_MD_command(source_plate='A', destination_plates=['B', 'B'], source_well=('A', 1), 
                            destination_wells= [('B', 1), ('C', 1)], source_volume=1000, transfer_volume=10, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
print(md_cmd)

MD,10
AspirateSpeed,2,5
DispenseSpeed,3,5
AIRGAP,5
TIP,1,1,0
MIX,2,3,70,4
REVERSE,5
Source,A,A-1,Source,Source,1000
Destination,B,B-1,Destination,Destination 
Destination,B,C-1,Destination,Destination 
MD



In [6]:
# test LT command:
lt_cmd = ezutils.generate_LT_command(source_plate='A', destination_plate='B', source_well=('A', 1), 
                            destination_well=('B', 1), source_volume=1000, transfer_volume=10, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
print(lt_cmd)

LT,10
AspirateSpeed,2,5
DispenseSpeed,3,5
AIRGAP,5
TIP,1,1,0
MIX,2,3,70,4
REVERSE,5
Source,A,A-1,Source,Source,1000
Destination,B,B-1,Destination,Destination 
LT


In [7]:
# generate sample MD file:
md_cmds = 'APM, 1\n'
for i in np.arange(1, 10):
    md_cmds += ezutils.generate_MD_command(source_plate='A', destination_plates=['B']*6, source_well=('A', i), 
                            destination_wells= [('A', 1), ('A', 2), ('B', 1), ('B', 2), ('C', 1), ('C', 2)], \
                            source_volume=1000, transfer_volume=10, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
print(md_cmds)
fd = open('MD_test.csv', 'w')
fd.write(md_cmds)
fd.close()

APM, 1
MD,10
AspirateSpeed,2,5
DispenseSpeed,3,5
AIRGAP,5
TIP,1,1,0
MIX,2,3,70,4
REVERSE,5
Source,A,A-1,Source,Source,1000
Destination,B,A-1,Destination,Destination 
Destination,B,A-2,Destination,Destination 
Destination,B,B-1,Destination,Destination 
Destination,B,B-2,Destination,Destination 
Destination,B,C-1,Destination,Destination 
Destination,B,C-2,Destination,Destination 
MD
MD,10
AspirateSpeed,2,5
DispenseSpeed,3,5
AIRGAP,5
TIP,1,1,0
MIX,2,3,70,4
REVERSE,5
Source,A,A-2,Source,Source,1000
Destination,B,A-1,Destination,Destination 
Destination,B,A-2,Destination,Destination 
Destination,B,B-1,Destination,Destination 
Destination,B,B-2,Destination,Destination 
Destination,B,C-1,Destination,Destination 
Destination,B,C-2,Destination,Destination 
MD
MD,10
AspirateSpeed,2,5
DispenseSpeed,3,5
AIRGAP,5
TIP,1,1,0
MIX,2,3,70,4
REVERSE,5
Source,A,A-3,Source,Source,1000
Destination,B,A-1,Destination,Destination 
Destination,B,A-2,Destination,Destination 
Destination,B,B-1,Destination,Destina

## Read pooling file (Shental format):

In [8]:
pooling_filename = 'pooling384_48_by_sample.txt'
pool_df = pd.read_csv(os.path.join(SAVE_PATH, pooling_filename), delimiter='\t', header=None)
pool_df.rename(columns={0: 'sample', 1: 'pool-1', 2: 'pool-2', 3: 'pool-3', 4: 'pool-4', 5: 'pool-5', 6: 'pool-6'}, inplace=True)
pool_df.drop(columns=[7], inplace=True)
pool_df.set_index('sample', inplace=True)
pool_df.head()

,pool-1,pool-2,pool-3,pool-4,pool-5,pool-6
sample,,,,,,
sample_1,1,9,17,25,33,41
sample_2,2,10,18,26,34,42
sample_3,3,11,19,27,35,43
sample_4,4,12,20,28,36,44
sample_5,5,13,21,29,37,45


## Generate mapping of samples to plates and plate mapsfrom samples in pool format 

In [15]:
import importlib
importlib.reload(ezutils)
plate_dict = {}
sample_df = pd.DataFrame(columns=['Plate', 'Row', 'Column'])

# write plate maps to excel sheet:
plate_writer = pd.ExcelWriter(path=os.path.join(SAVE_PATH, 'sample_plate_maps_384_format.xlsx'))

for plate_ind, sample_inds in zip([1, 2, 3, 4], 
                                  (np.arange(0, 96), np.arange(96, 192), np.arange(192, 288), np.arange(288, 384))):
    curr_plate_df, curr_sample_df = ezutils.sample_list_to_plate_map(sample_list=pool_df.index[sample_inds],
                                                                     plate_num=plate_ind, plate_format='96_well')
    curr_plate_df.name = plate_ind

    plate_dict[plate_ind] = curr_plate_df
    curr_plate_df.to_excel(plate_writer, sheet_name='plate ' + str(plate_ind))
    sample_df = sample_df.append(curr_sample_df)
    
plate_writer.close()

display(sample_df.tail())
plate_dict

,Plate,Row,Column
sample_380,4,H,8
sample_381,4,H,9
sample_382,4,H,10
sample_383,4,H,11
sample_384,4,H,12


{1:           1          2          3          4          5          6   \
 A   sample_1   sample_2   sample_3   sample_4   sample_5   sample_6   
 B  sample_13  sample_14  sample_15  sample_16  sample_17  sample_18   
 C  sample_25  sample_26  sample_27  sample_28  sample_29  sample_30   
 D  sample_37  sample_38  sample_39  sample_40  sample_41  sample_42   
 E  sample_49  sample_50  sample_51  sample_52  sample_53  sample_54   
 F  sample_61  sample_62  sample_63  sample_64  sample_65  sample_66   
 G  sample_73  sample_74  sample_75  sample_76  sample_77  sample_78   
 H  sample_85  sample_86  sample_87  sample_88  sample_89  sample_90   
 
           7          8          9          10         11         12  
 A   sample_7   sample_8   sample_9  sample_10  sample_11  sample_12  
 B  sample_19  sample_20  sample_21  sample_22  sample_23  sample_24  
 C  sample_31  sample_32  sample_33  sample_34  sample_35  sample_36  
 D  sample_43  sample_44  sample_45  sample_46  sample_47  samp

## Merge the location of each sample into pool_df

In [16]:
sample_master_df = pool_df.join(sample_df)
sample_master_df.rename(columns={'Row': 'Sample_plate_Row', 'Column': 'Sample_plate_Column', 'Plate': 'Sample_Plate'}, inplace=True)
sample_master_df.head()



,pool-1,pool-2,pool-3,pool-4,pool-5,pool-6,Sample_Plate,Sample_plate_Row,Sample_plate_Column
sample,,,,,,,,,
sample_1,1,9,17,25,33,41,1,A,1
sample_2,2,10,18,26,34,42,1,A,2
sample_3,3,11,19,27,35,43,1,A,3
sample_4,4,12,20,28,36,44,1,A,4
sample_5,5,13,21,29,37,45,1,A,5


## Generate pool map from pool names:

In [17]:
# generate 48 pools:
pool_names  = pd.Index(['pool_' + str(i) for i in range(1,49)]) 

In [18]:
pool_plate_df, pool_location_df = ezutils.sample_list_to_plate_map(sample_list=pool_names, plate_num=1, plate_format='48_well')

writer = pd.ExcelWriter(path=os.path.join(SAVE_PATH, 'pool_locations.xlsx'))
pool_plate_df.to_excel(writer, sheet_name='pool_plate_map')
pool_location_df.to_excel(writer, sheet_name='pool_locations')
writer.close()
display(pool_plate_df)
display(pool_location_df.head())

,1,2,3,4,5,6,7,8
A,pool_1,pool_2,pool_3,pool_4,pool_5,pool_6,pool_7,pool_8
B,pool_9,pool_10,pool_11,pool_12,pool_13,pool_14,pool_15,pool_16
C,pool_17,pool_18,pool_19,pool_20,pool_21,pool_22,pool_23,pool_24
D,pool_25,pool_26,pool_27,pool_28,pool_29,pool_30,pool_31,pool_32
E,pool_33,pool_34,pool_35,pool_36,pool_37,pool_38,pool_39,pool_40
F,pool_41,pool_42,pool_43,pool_44,pool_45,pool_46,pool_47,pool_48


,Plate,Row,Column
pool_1,1,A,1
pool_2,1,A,2
pool_3,1,A,3
pool_4,1,A,4
pool_5,1,A,5


In [19]:
def pool_num_to_pool_plate_location(pool_num, pool_location_df):
    p, r, c = pool_location_df.loc['pool_' + str(pool_num)]
    #return 'plate ' + str(p) + ': ' + r + '-' + str(c)
    return r + '-' + str(c)
pool_num_to_pool_plate_location(2, pool_location_df)

'A-2'

In [20]:
for p in ['pool-1', 'pool-2', 'pool-3', 'pool-4', 'pool-5', 'pool-6']:
    sample_master_df.loc[:, p + '_location'] = sample_master_df[p].apply(pool_num_to_pool_plate_location, args=(pool_location_df,))

In [21]:
sample_master_df

,pool-1,pool-2,pool-3,pool-4,pool-5,pool-6,Sample_Plate,Sample_plate_Row,Sample_plate_Column,pool-1_location,pool-2_location,pool-3_location,pool-4_location,pool-5_location,pool-6_location
sample,,,,,,,,,,,,,,,
sample_1,1,9,17,25,33,41,1,A,1,A-1,B-1,C-1,D-1,E-1,F-1
sample_2,2,10,18,26,34,42,1,A,2,A-2,B-2,C-2,D-2,E-2,F-2
sample_3,3,11,19,27,35,43,1,A,3,A-3,B-3,C-3,D-3,E-3,F-3
sample_4,4,12,20,28,36,44,1,A,4,A-4,B-4,C-4,D-4,E-4,F-4
sample_5,5,13,21,29,37,45,1,A,5,A-5,B-5,C-5,D-5,E-5,F-5
sample_6,6,14,22,30,38,46,1,A,6,A-6,B-6,C-6,D-6,E-6,F-6
sample_7,7,15,23,31,39,47,1,A,7,A-7,B-7,C-7,D-7,E-7,F-7
sample_8,8,16,24,32,40,48,1,A,8,A-8,B-8,C-8,D-8,E-8,F-8
sample_9,1,10,19,28,37,46,1,A,9,A-1,B-2,C-3,D-4,E-5,F-6


## Write sample_to_pool_and_plate to excel sheet:

In [22]:
# write sample_to_pool file to excel sheet:
writer = pd.ExcelWriter(path=os.path.join(SAVE_PATH, 'sample_to_pools_and_plates_384_to48_format.xlsx'))
sample_master_df.to_excel(writer)
writer.close()

## Generate EZMate commands from sample_master_df

In [23]:
def sample_ind_to_ezmate_plate(sample_ind, plate_format='96_well'):
    """
    assumes that robot uses two source plates for pooling - A and B and needs to figure out which to use for each sample.
    if sample in first 96 A, if second C. works on 384 well format for now.
    """
    if plate_format == '96_well':

        if sample_ind <= 96:
            return 'A'
        if (sample_ind > 96) and  (sample_ind <=192):
            return 'C'
        if (sample_ind > 192) and (sample_ind <= 288):
            return 'A'
        if sample_ind > 288:
            return 'C'
    else:
        error('no such plate')

In [24]:
sample_master_df.head()

,pool-1,pool-2,pool-3,pool-4,pool-5,pool-6,Sample_Plate,Sample_plate_Row,Sample_plate_Column,pool-1_location,pool-2_location,pool-3_location,pool-4_location,pool-5_location,pool-6_location
sample,,,,,,,,,,,,,,,
sample_1,1,9,17,25,33,41,1,A,1,A-1,B-1,C-1,D-1,E-1,F-1
sample_2,2,10,18,26,34,42,1,A,2,A-2,B-2,C-2,D-2,E-2,F-2
sample_3,3,11,19,27,35,43,1,A,3,A-3,B-3,C-3,D-3,E-3,F-3
sample_4,4,12,20,28,36,44,1,A,4,A-4,B-4,C-4,D-4,E-4,F-4
sample_5,5,13,21,29,37,45,1,A,5,A-5,B-5,C-5,D-5,E-5,F-5


'/Users/thertz/Dropbox/HertzLab/Code/Python/COVID-19'

In [26]:
os.path.join(SAVE_PATH, 'EZMate_384_to_48_pools_MD_commands.csv')

'/home/tomer/Dropbox/HertzLab/Code/Python/COVID-19/EZMate_384_to_48_pools_MD_commands.csv'

### Run script for first 192 samples only - plates 1 AND 2

In [120]:
# generate sample MD file:
md_cmds = 'APM, 1\n'
for s in sample_master_df[0:192].index:
    
    sample_ind = int(s.split('_')[-1])
    source_plate = sample_ind_to_ezmate_plate(sample_ind)
    
    source_well = (sample_master_df.loc[s]['Sample_plate_Row'], sample_master_df.loc[s]['Sample_plate_Column'])
    
    # due to AIRGAP and for more consistent pippeting, first dispense back into original pool
    destination_wells = [source_well]
    for d in range(1,7):
        curr_col = 'pool-' + str(d) + '_location'
        destination_wells.append((sample_master_df.loc[s][curr_col].split('-')[0], sample_master_df.loc[s][curr_col].split('-')[1]) )
    
    destination_plates = [source_plate] + ['C']*6
    
    md_cmds += ezutils.generate_MD_command(source_plate=source_plate, destination_plates=destination_plates, source_well=source_well, 
                            destination_wells= destination_wells, source_volume=100, transfer_volume=11, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
fd = open(os.path.join(SAVE_PATH, 'EZMate_384_to_48_pools_MD_commands_plates_1_2.csv'), 'w')
fd.write(md_cmds)
fd.close()

### Run script for second 192 samples only - plates 3 AND 4

In [121]:
# generate sample MD file:
md_cmds = 'APM, 1\n'
for s in sample_master_df[192:].index:
    
    sample_ind = int(s.split('_')[-1])
    source_plate = sample_ind_to_ezmate_plate(sample_ind)
    
    source_well = (sample_master_df.loc[s]['Sample_plate_Row'], sample_master_df.loc[s]['Sample_plate_Column'])
    
    # due to AIRGAP and for more consistent pippeting, first dispense back into original pool
    destination_wells = [source_well]
    for d in range(1,7):
        curr_col = 'pool-' + str(d) + '_location'
        destination_wells.append((sample_master_df.loc[s][curr_col].split('-')[0], sample_master_df.loc[s][curr_col].split('-')[1]) )
    
    destination_plates = [source_plate] + ['C']*6
    
    md_cmds += ezutils.generate_MD_command(source_plate=source_plate, destination_plates=destination_plates, source_well=source_well, 
                            destination_wells= destination_wells, source_volume=100, transfer_volume=11, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
fd = open(os.path.join(SAVE_PATH, 'EZMate_384_to_48_pools_MD_commands_plates_3_4.csv'), 'w')
fd.write(md_cmds)
fd.close()

### Code for making two pools sets simultaneously

In [122]:
# generate sample MD file:
md_cmds = 'APM, 1\n'
for s in sample_master_df[0:192].index:
    
    sample_ind = int(s.split('_')[-1])
    source_plate = sample_ind_to_ezmate_plate(sample_ind)
    
    source_well = (sample_master_df.loc[s]['Sample_plate_Row'], sample_master_df.loc[s]['Sample_plate_Column'])
    
    # due to AIRGAP and for more consistent pippeting, first dispense back into original pool
    destination_wells = [source_well]
    for d in range(1,7):
        curr_col = 'pool-' + str(d) + '_location'
        destination_wells.append((sample_master_df.loc[s][curr_col].split('-')[0], sample_master_df.loc[s][curr_col].split('-')[1]) )
    
    destination_wells = destination_wells + destination_wells[1:]
    destination_plates = [source_plate] + ['C']*6 + ['D']*6
    
    md_cmds += ezutils.generate_MD_command(source_plate=source_plate, destination_plates=destination_plates, source_well=source_well, 
                            destination_wells= destination_wells, source_volume=100, transfer_volume=11, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
fd = open(os.path.join(SAVE_PATH, 'EZMate_384_to_48_pools_MD_commands_plates_1_2_duplicate_pools.csv'), 'w')
fd.write(md_cmds)
fd.close()

### Run script for second 192 samples only - plates 3 AND 4

In [123]:
# generate sample MD file:
md_cmds = 'APM, 1\n'
for s in sample_master_df[192:].index:
    
    sample_ind = int(s.split('_')[-1])
    source_plate = sample_ind_to_ezmate_plate(sample_ind)
    
    source_well = (sample_master_df.loc[s]['Sample_plate_Row'], sample_master_df.loc[s]['Sample_plate_Column'])
    
    # due to AIRGAP and for more consistent pippeting, first dispense back into original pool
    destination_wells = [source_well]
    for d in range(1,7):
        curr_col = 'pool-' + str(d) + '_location'
        destination_wells.append((sample_master_df.loc[s][curr_col].split('-')[0], sample_master_df.loc[s][curr_col].split('-')[1]) )
    
    
    destination_wells = destination_wells + destination_wells[1:]
    destination_plates = [source_plate] + ['C']*6 + ['D']*6
    
    md_cmds += ezutils.generate_MD_command(source_plate=source_plate, destination_plates=destination_plates, source_well=source_well, 
                            destination_wells= destination_wells, source_volume=100, transfer_volume=11, 
                            aspirate_speed=(2, 5), dispense_speed=(3, 5), air_gap=5, tip=(1, 1, 0), mix=(2, 3, 70, 4),
                            reverse=5)
fd = open(os.path.join(SAVE_PATH, 'EZMate_384_to_48_pools_MD_commands_plates_3_4_duplicate_pools.csv'), 'w')
fd.write(md_cmds)
fd.close()

In [126]:
filename = '/Users/thertz/Downloads/48_pools_with_384_samples.xlsx'
df = pd.read_excel(filename, sheet_name='Sheet2')
df.head()

,pool 4,pool 7,pool 9,pool 13,pool 19,pool 22,pool 26,pool 32,pool 33,pool 44
0,4,7,1,5,3,6,2,8,1,4
1,12,15,10,14,9,16,11,13,13,15
2,20,23,19,23,23,18,24,18,20,19
3,28,31,28,32,29,28,29,27,32,32
4,36,39,37,33,34,39,39,33,39,34


In [156]:
curr_inds = pd.unique(df[df.columns].values.ravel('K'))
sample_counts = []
for i in curr_inds:
    sample_counts.append(df[df == i].count().sum())
    
max_ind = np.argmax(sample_counts)
max_sample = curr_inds[max_ind]
print('sample is: {0}, count is: {1}'.format(max_sample, sample_counts[max_ind]))

np.argsort(sample_counts)

print('sample is: {0}, count is: {1}'.format(curr_inds[64], 5))


sample is: 140, count is: 6
sample is: 135, count is: 5


In [132]:
for i in df.values.unique()
df[df == 4.0].count().sum()

2

In [ ]:
# dest plae for pools is 6x8 A-F, 1-8, source plates are 96-well A-H 1-12.
# generate 96 well plate map, generate new pool file in which each sample has a plate/loaction and each pool is a plate/location
